# **LlamaIndex** combined with **LangChain** to use GPT4All Model

*This notebook was created on **22/07/2024** using python 3.12 version.*

In this notebook we will use `llama_index` library combined with `langchain` to load a **GPT4All** model on your computer, and use RUG to ask question to the model on topics that where not part of the training set of the model itself.

For this example, we are going to use the model **SBert** (*all-MiniLM-L6-v2.gguf2.f16.gguf*) as embedding model, and model **GPT4All Falcon** (*gpt4all-falcon-newbpe-q4_0.gguf*) for querying our documents. \
Observe that for this model is suggested a 8 GB RAM computer.

##### Remark

Before running this tutorial is suggested to read the previous Jupyter Notebooks:

1. gpt4all/gpt4all-model-list.ipynb
2. gpt4all/gpt4all-base-model.ipynb
3. langchain/langchain-gpt4all-base-model

In [ ]:
%pip install gpt4all==2.7.0
%pip install llama-index==0.10.56
%pip install llama-index-llms-langchain==0.1.4
%pip install llama-index-embeddings-langchain==0.1.2

In [1]:
# Import libraries
from pathlib import Path
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.langchain import LangChainLLM
from langchain_community.llms.gpt4all import GPT4All

from gpt4all import Embed4All
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings

In [2]:
# Load the input text
documents = SimpleDirectoryReader("../data").load_data()

In [4]:
# Set model path and name
model_path = Path().home() / ".data" / "model"
if not model_path.exists():
    raise Exception(f"The path '{model_path}' does not exists!")

embedding_model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
embedding_model_path_name = model_path / embedding_model_name

model_name = "gpt4all-falcon-newbpe-q4_0.gguf"
model_path_name = model_path / model_name

In [5]:
# Load gpt4all model
model = GPT4All(model = str(model_path_name))
print("Model loaded!")

Model loaded!


In [6]:
# Create LLM using Llama Index
llm = LangChainLLM(llm = model)
print("LLM created!")

LLM created!


In [8]:
# Create Document Index
gpt4all_embedding = GPT4AllEmbeddings(client = Embed4All(model_name = str(embedding_model_path_name)))
embed_model = LangchainEmbedding(langchain_embeddings = gpt4all_embedding)
index = VectorStoreIndex.from_documents(documents, embed_model = embed_model, show_progress = True)
print("Index created!")

/Users/andreazanini/.pyenv/versions/3.12.0/envs/py312gpt4all/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  3.50it/s]

Index created!


In [9]:
# Create Query Engine
query_engine = index.as_query_engine(llm = llm)
print("Query Engine created!")

Query Engine created!


In [12]:
# Ask question
response = query_engine.query("Where are Lydia and Andrew going during the night?")
print(response)

"Lydia and Andrew are going deep into the Forbidden Forest to find a hidden creature that could be incredibly important for Andrew's research."
